In [1]:
import os
import streamlit as st

os.environ["HUGGINGFACEHUB_API_TOKEN"] = st.secrets["env"]["HUGGINGFACEHUB_API_TOKEN"]

In [2]:
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [3]:
# from langchain.document_loaders import PyPDFLoader
# def load_pdf_files(files):
#     # Assuming the provided 'files' list contains URLs
#     pdf_url = files[0]

#     # Read PDF content from the URL
#     pdf_loader = PyPDFLoader(pdf_url)
#     docs = pdf_loader.load_and_split()

#     # If there are more files in the list, process them
#     for i in range(1, len(files)):
#         pdf_url = files[i]
#         pdf_loader = PyPDFLoader(pdf_url)
#         docs += pdf_loader.load_and_split()

#     return docs

In [4]:
# files=["./DataSourceFiles/IPC_186045.pdf"]
# # files=["D:/Files/LLM/Project/DataSourceFiles/IPC_186045.pdf","D:\Files\LLM\Src_docs\special_marriage_act.pdf","D:\Files\LLM\Src_docs\THE_CODE_OF_CIVIL_PROCEDURE_1908.pdf","D:\Files\LLM\Src_docs\THE_LAND_ACQUISITION_ACT.pdf"]
# docs=load_pdf_files(files)
# print("docs in PDF files: ",len(docs))
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("DataSourceFiles\IPC_186045.pdf",extract_images=True)
pages = loader.load_and_split()

In [5]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# def split_docs(documents,chunk_size=1000,chunk_overlap=20):
#   text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#   docs = text_splitter.split_documents(documents)
#   return docs

# splited_docs = split_docs(docs)
# print(len(splited_docs))

In [6]:
from langchain.embeddings import  HuggingFaceInstructEmbeddings,HuggingFaceHubEmbeddings,HuggingFaceEmbeddings
HFIembeddings = HuggingFaceInstructEmbeddings(model_name="thenlper/gte-small",cache_folder="./Models/", model_kwargs={"device": DEVICE})

d:\Files\LLM\Project\venv\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
2024-02-11 18:57:27.114 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: thenlper/gte-small


load INSTRUCTOR_Transformer
max_seq_length  512


In [16]:
from langchain.vectorstores import Chroma

# vectorStoreDB = Chroma.from_documents(documents=pages,embedding=HFIembeddings)
# del vectorStoreDB
vectorStoreDB = Chroma.from_documents(documents=pages,embedding=HFIembeddings,persist_directory="db")

ValueError: An instance of Chroma already exists for ephemeral with different settings

In [ ]:
# docs = vectorStoreDB.similarity_search("who is judge?")


In [17]:
# # persiste the db to disk
# vectorStoreDB.persist()
# vectordb = None

ValueError: You must specify a persist_directory oncreation to persist the collection.

In [19]:
vectorstore = Chroma(persist_directory="db", embedding_function=HFIembeddings)

ValueError: An instance of Chroma already exists for ephemeral with different settings

In [ ]:
# from langchain.embeddings import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(
#     model_name="D:\Files\LLM\Project\Models\sentence-transformers_all-MiniLM-L6-v2",
#     model_kwargs={'device': 'cuda'},
#     encode_kwargs={'normalize_embeddings': True},
#     # query_instruction="Represent this sentence for searching relevant passages: "
# )

# from langchain.vectorstores import Chroma
# vectorstore = Chroma.from_documents(splited_docs, embeddings)

In [ ]:
# from langchain.vectorstores import Chroma
# # from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# # from langchain.embeddings import  HuggingFaceInstructEmbeddings,HuggingFaceHubEmbeddings,HuggingFaceEmbeddings

# from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
# from sentence_transformers import SentenceTransformer
# # model = SentenceTransformer(model_name_or_path="sentence-transformers/all-MiniLM-L6-v2",cache_folder="./Models/")
# sentence_transformer_ef = SentenceTransformerEmbeddingFunction(model_name="D:\Files\LLM\Project\Models\sentence-transformers_all-MiniLM-L6-v2")
# # # HFIembeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/all-mpnet-base-v2")
# # # # embedding_function = SentenceTransformerEmbeddings(model_name="gte-small") #,cache_folder="./Models/")
# # print((sentence_transformer_ef))
# vectorstore = Chroma.from_documents(documents=pages,embedding=sentence_transformer_ef,collection_name="law_collection")

In [ ]:
from langchain.prompts import PromptTemplate
template = """
    - Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer
    - You're a helpful AI assistant assigned to assist individuals seeking legal advice within the framework of Indian laws and the constitution.
    - Your role is to guide users through legal processes and provide information in a lawful manner.
    - Answer questions step by step, highlighting relevant sections of Indian laws and the constitution.
    - Refrain from responding to queries that may not contribute to legal affairs, and provide accurate and relevant information without distortion.
{context}

Question: {question} 
Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
chain = RetrievalQA.from_chain_type(
    llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={"temperature":0.8, "max_length":4096,"max_new_tokens":4096}),
    chain_type="stuff",
    retriever = vectorStoreDB.as_retriever(search_kwargs={"k": 7}),    
    # return_source_documents=True,
    chain_type_kwargs={"prompt": prompt} #,"verbose": True},
)

In [ ]:
# q = "what is the punishment for robbery ?"
# q = "who is judge ?"
# q = "one person has commited defamation, what should be punishment for it?"
# q = "rioting has happened in neighbouring society. who will be responsible and what is the punishment for it"
q = "what is punishment for robbery and also murder ?"
# q = "what are the rights of tenant and landlord ?"
# q = "Liability of person for whose benefit riot is committed"
# q = "person is selling adulterated drugs which is harmful to the health of people. person has also sold drug to the children below age of 15 what will be punishment for it?"
# q="start with python programming language"
# q="explain all laws related to companies"
response = chain({"query":q, "early_stopping":True,"min_length":2000,"max_tokens":5000})

from write_in_file import generate_docx_with_bullets
# from data_extracter import extract_data_from_response
# result,src_data,src_pg_nms=extract_data_from_response(response)
# generate_docx_with_bullets(heading=q,main_paragraph=response["result"],srcs=src_pg_nms,output_folder="./tmp/")
generate_docx_with_bullets(heading=q,main_paragraph=response["result"],output_folder="./tmp/")

Document './tmp/2024-02-11_18_35_33.docx' generated successfully.
